In [ ]:
import pandas as pd
import geopandas as gpd
import glob
from pathlib import Path

### Operator statistics
This notebooks generates information on the operators, their parcel count, etc.

In [ ]:
# Inputs and outputs
results_path = Path("../results")
tables_path = Path("tables")

In [ ]:
# Load operator data per depot
df_operators = []

for path in glob.glob(str(results_path / "parcels/depots/today_2024/*.gpkg")):
    df_depot = gpd.read_file(path)

    df_operators.append({
        "operator": df_depot["operator"].values[0],
        "depots": len(df_depot["depot_id"].unique()),
        "parcels": len(df_depot["parcel_id"].unique())
    })

df_operators = pd.DataFrame.from_records(df_operators)
df_operators = df_operators.sort_values(by = ["parcels"], ascending = False)

df_operators = pd.concat([
    df_operators, pd.DataFrame({
        "operator": ["SUM"], "depots": [df_operators["depots"].sum()], "parcels": [df_operators["parcels"].sum()]
    })
])

In [ ]:
# Prepare table
df_table = df_operators.copy()
df_table["operator"] = df_table["operator"].replace({
    "laposte": "La Poste (Colissimo)", "chronopost": "Chronopost", "ups": "UPS", "dpd": "DPD",
    "dhl": "DHL", "gls": "GLS", "colisprive": "Colis privé", "fedex": "Fedex",
    "SUM": "$\\sum$"
})
df_table.columns = ["Operator", "Depots", "Parcels"]
df_table = df_table.style.format(thousands = ",").hide(axis = "index")

with open(tables_path / "depots.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "Operator statistics for 2024", label = "tab:depots", hrules = True)
    table = table.replace("$\\sum$", "\\midrule\n$\\sum$")
    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")
    f.write(table)

df_table